# Hyperparameter Tuning using HyperDrive

In [2]:
import os
import sys
import json
import azureml
import logging
import requests
import pandas as pd
import numpy as np
from io import BytesIO
import joblib

from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.core import Environment

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.estimator import Estimator
from azureml.core import ScriptRunConfig
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

from azureml.core.dataset import Dataset
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling, BayesianParameterSampling
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, quniform, choice

from azureml.core.runconfig import RunConfiguration
from azureml.core.runconfig import EnvironmentDefinition
from azureml.core.runconfig import CondaDependencies
from azureml.core.model import InferenceConfig, Model

from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model, InferenceConfig

from azureml.train.automl import constants

from train import *

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', None)

# Check system and core SDK version number
print("System version: {}".format(sys.version))
print("SDK version:", azureml.core.VERSION)

System version: 3.6.13 |Anaconda, Inc.| (default, Feb 23 2021, 12:58:59) 
[GCC Clang 10.0.0 ]
SDK version: 1.23.0


In [3]:
interactive_auth = InteractiveLoginAuthentication(tenant_id="660b3398-b80e-49d2-bc5b-ac1dc93b5254")
ws = Workspace(subscription_id="f5091c60-1c3c-430f-8d81-d802f6bf2414",
                   resource_group="aml-quickstarts-142350",
                   workspace_name="quick-starts-ws-142350",
                   auth=interactive_auth)

experiment_name = 'online_news_project'
experiment=Experiment(ws, experiment_name)
experiment

# ws = Workspace.from_config()
# print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Name,Workspace,Report Page,Docs Page
online_news_project,quick-starts-ws-142350,Link to Azure Machine Learning studio,Link to Documentation


In [4]:
dic_data = {'Workspace name': ws.name,
            'Azure region': ws.location,
            'Subscription id': ws.subscription_id,
            'Resource group': ws.resource_group,
            'Experiment Name': experiment.name}

az_data = pd.DataFrame.from_dict(data = dic_data, orient='index')
az_data.rename(columns={0:''}, inplace = True)
az_data

,
Workspace name,quick-starts-ws-142350
Azure region,southcentralus
Subscription id,f5091c60-1c3c-430f-8d81-d802f6bf2414
Resource group,aml-quickstarts-142350
Experiment Name,online_news_project


## Create or Attach an AmlCompute cluster

In [5]:
# Define CPU cluster name
compute_target_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=compute_target_name)
    print("Found existing cpu-cluster. Use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_DS12_V2",
                                                           min_nodes=1, 
                                                           max_nodes=4) 
    compute_target = ComputeTarget.create(ws, compute_target_name, compute_config)

compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())

Found existing cpu-cluster. Use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 3, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 1, 'unusableNodeCount': 0, 'leavingNodeCount': 2, 'preemptedNodeCount': 0}, 'allocationState': 'Resizing', 'allocationStateTransitionTime': '2021-04-11T01:44:29.130000+00:00', 'errors': None, 'creationTime': '2021-04-10T23:22:27.803998+00:00', 'modifiedTime': '2021-04-10T23:22:43.243621+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS12_V2'}


## Dataset

The dataset used in this project is a dataset made available on UCI Machine Learning Repository called [Online News Popularity Data Set](https://archive.ics.uci.edu/ml/datasets/Online+News+Popularity#).

The dataset summarizes heterogeneous set of features about the articles published by Mashable between 2013 and 2015.

- Number of Instances: 39797
- Number of Attributes: 61 
    - 58 predictive attributes 
    - 2 non-predictive (`url` and `timedelta`) 
    - 1 target column
    
We will also apply the `Boruta` step for feature selection before exporting the data to our defined Datastore

In [6]:
DATA_LOC = "https://raw.githubusercontent.com/franckess/AzureML_Capstone/main/data/OnlineNewsPopularity.csv"
BORUTA_LOC = "https://github.com/franckess/AzureML_Capstone/releases/download/1.1/boruta_model_final.pkl"

# Loading data
df = pd.read_csv(DATA_LOC)

# Removing space character in the feature names
df.columns=df.columns.str.replace(' ','')

# Drop URL column
df = df.drop(['url'], axis=1)

# Perform Data pre-processing
df = corr_drop_cols(df)
df = create_label(df)
df = scaling_num(df)
df = feature_selection(df, BORUTA_LOC)
    
# Split train data into train & test
X_train, X_test, y_train, y_test = split_train_test(df)

m, k = X_train.shape
print("{} x {} table of data:".format(m, k))
X_train.info()

31715 x 47 table of data:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 31715 entries, 38512 to 35050
Data columns (total 47 columns):
n_tokens_title                   31715 non-null float64
n_tokens_content                 31715 non-null float64
n_unique_tokens                  31715 non-null float64
num_hrefs                        31715 non-null float64
num_self_hrefs                   31715 non-null float64
num_imgs                         31715 non-null float64
num_videos                       31715 non-null float64
average_token_length             31715 non-null float64
num_keywords                     31715 non-null float64
data_channel_is_entertainment    31715 non-null int64
data_channel_is_bus              31715 non-null int64
data_channel_is_socmed           31715 non-null int64
data_channel_is_tech             31715 non-null int64
data_channel_is_world            31715 non-null int64
kw_min_min                       31715 non-null float64
kw_max_min                     

## Upload data to Azure Datatore

In [ ]:
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./data/OnlineNewsPopularity.csv'], target_path='data/', overwrite=True, show_progress=True)

In [ ]:
print(
    "Datastore type: " + datastore.datastore_type,
    "Account name: " + datastore.account_name,
    "Container name: " + datastore.container_name,
    sep="\n",
)

In [ ]:
# Get data reference object for the data path
ds_data = datastore.path('data/')
print(ds_data)

## HyperDrive Configuration

### Create an environment

Define a conda environment YAML file with your training script dependencies and create an Azure ML environment.

In [17]:
%%writefile hyperdrive_dependencies.yml

channels:
- conda-forge
- anaconda
dependencies:
- python=3.6.2
- scikit-learn
- pandas
- numpy
- pip:
  - azureml-defaults 
  - pingouin
  - lightgbm
  - joblib  
  - Boruta
  - inference-schema
  - azureml-interpret
  - azureml-train-automl-runtime

Overwriting hyperdrive_dependencies.yml


In [18]:
udacity_env = Environment.from_conda_specification(name = 'udacity-env', file_path = './hyperdrive_dependencies.yml')
udacity_env.register(ws)

# Specify an Ubuntu base image
udacity_env.docker.enabled = True
udacity_env.python.user_managed_dependencies = False
udacity_env.docker.base_image = 'mcr.microsoft.com/azureml/base:openmpi3.1.2-ubuntu18.04'

Build the image just to confirm it works appropriately or identify any errors prior to deployment

In [ ]:
# build = udacity_env.build(workspace=ws)
# build.wait_for_completion(show_output=True)

In [ ]:
# details = udacity_env.get_image_details(ws)
# print(details.dockerfile)

### Tune hyperparameters using `HyperDrive`

In the following block, I tune my set of hyperparameters for the `LightGBM` model. The ranges of parameters for the `LightGBM` used were chosen considering the parameters tuning guides for different scenarios provided here.

The code below does a parallel search of the hyperparameter space using a Bayesian sampling method which does not support termination policy. Therefore, `policy=None`.

> __Note that when using Bayesian sampling, the number of concurrent runs has an impact on the effectiveness of the tuning process. Typically, a smaller number of concurrent runs leads to better sampling convergence. That is because some runs start without fully benefiting from runs that are still running.__

In order to compare the performance of HyperDrive with the one of AutoML we chose as objective metric of `LightGBM` __Accuracy__ score.

In [19]:
# Create a SKLearn estimator for use with train.py
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      environment=udacity_env)

# Specify a Policy
#early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2, delay_evaluation=5)

# Specify hyperparameter space
param_sampling = BayesianParameterSampling(
    {
        "--num-leaves": quniform(8, 128, 1),
        "--min-data-in-leaf": quniform(20, 500, 10),
        "--learning-rate": choice(
            1e-4, 1e-3, 5e-3, 1e-2, 1.5e-2, 2e-2, 3e-2, 5e-2, 1e-1
        ),
        "--feature-fraction": uniform(0.1, 1),
        "--bagging-fraction": uniform(0.1, 1),
        "--bagging-freq": quniform(1, 30, 1),
        "--max-depth": quniform(5, 50, 5)
    }
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=param_sampling,
                                     policy=None ,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=50,
                                     max_concurrent_runs=10)

In [20]:
# Submit hyperdrive run to the experiment 
hyperdrive_run = experiment.submit(config=hyperdrive_config, show_output=True)

## Run Details

In [21]:
# Show run details with the Jupyter widget
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
hyperdrive_run.get_metrics()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_74fb10e2-6617-418a-87ba-0fe657f41206
Web View: https://ml.azure.com/experiments/online_news_project/runs/HD_74fb10e2-6617-418a-87ba-0fe657f41206?wsid=/subscriptions/f5091c60-1c3c-430f-8d81-d802f6bf2414/resourcegroups/aml-quickstarts-142350/workspaces/quick-starts-ws-142350

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-04-11T02:19:40.734091][API][INFO]Experiment created<END>\n""<START>[2021-04-11T02:19:41.514124][GENERATOR][INFO]Trying to sample '10' jobs from the hyperparameter space<END>\n""<START>[2021-04-11T02:19:42.025826][GENERATOR][INFO]Successfully sampled '10' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_74fb10e2-6617-418a-87ba-0fe657f41206
Web View: https://ml.azure.com/experiments/online_news_project/runs/HD_74fb10e2-6617-418a-87ba-0fe657f41206?wsid=/subscriptions/f5091c60-1c3c-430f-8d81-d802f6bf2414/resourcegroups/aml-quickstarts-142350/workspaces/quick-starts-ws-142350



{'HD_74fb10e2-6617-418a-87ba-0fe657f41206_49': {'Accuracy': 0.677639046538025},
 'HD_74fb10e2-6617-418a-87ba-0fe657f41206_48': {'Accuracy': 0.6757472569050321},
 'HD_74fb10e2-6617-418a-87ba-0fe657f41206_47': {'Accuracy': 0.6739815865809056},
 'HD_74fb10e2-6617-418a-87ba-0fe657f41206_46': {'Accuracy': 0.533610795812839},
 'HD_74fb10e2-6617-418a-87ba-0fe657f41206_44': {'Accuracy': 0.6501450372051961},
 'HD_74fb10e2-6617-418a-87ba-0fe657f41206_45': {'Accuracy': 0.6762517341404969},
 'HD_74fb10e2-6617-418a-87ba-0fe657f41206_43': {'Accuracy': 0.6505233951317947},
 'HD_74fb10e2-6617-418a-87ba-0fe657f41206_40': {'Accuracy': 0.6761256148316307},
 'HD_74fb10e2-6617-418a-87ba-0fe657f41206_42': {'Accuracy': 0.6741077058897718},
 'HD_74fb10e2-6617-418a-87ba-0fe657f41206_41': {'Accuracy': 0.6732248707277084},
 'HD_74fb10e2-6617-418a-87ba-0fe657f41206_39': {'Accuracy': 0.6791524782444193},
 'HD_74fb10e2-6617-418a-87ba-0fe657f41206_38': {'Accuracy': 0.6741077058897718},
 'HD_74fb10e2-6617-418a-87ba-0

## Retrieve and Save Best Model

Here I retrieve and save the best model as well as display all the properties of the model.

In [22]:
get_best_performance = hyperdrive_run.get_best_run_by_primary_metric() 
get_best_metrics = get_best_performance.get_metrics()
parameter_values = get_best_performance.get_details()["runDefinition"]["arguments"]

print('Best Run Id: ', get_best_performance.id)
print('Best Run Accuracy:', get_best_metrics['Accuracy'])
print('Best model hyperparameter values', parameter_values)

Best Run Id:  HD_74fb10e2-6617-418a-87ba-0fe657f41206_29
Best Run Accuracy: 0.6800353134064825
Best model hyperparameter values ['--num-leaves', '50', '--min-data-in-leaf', '350', '--learning-rate', '0.03', '--feature-fraction', '0.9010982437467641', '--bagging-fraction', '1', '--bagging-freq', '26', '--max-depth', '25']


In [23]:
get_best_performance.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_af281b0470a9de5e88951593a7cb59c2a95b736aefc5aeb7831a2b61e6aa5f20_d.txt',
 'azureml-logs/65_job_prep-tvmps_af281b0470a9de5e88951593a7cb59c2a95b736aefc5aeb7831a2b61e6aa5f20_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_af281b0470a9de5e88951593a7cb59c2a95b736aefc5aeb7831a2b61e6aa5f20_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/97_azureml.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/lgb_model.pkl']

In [35]:
model = get_best_performance.register_model(model_name = 'best_model', 
                                            model_path = './outputs/lgb_model.pkl')
print("Model successfully registered.")

Model successfully registered.


In [36]:
model

Model(workspace=Workspace.create(name='quick-starts-ws-142350', subscription_id='f5091c60-1c3c-430f-8d81-d802f6bf2414', resource_group='aml-quickstarts-142350'), name=best_model, id=best_model:1, version=1, tags={}, properties={})

In [25]:
get_best_performance.download_file("outputs/lgb_model.pkl","output/hyperdrive_model.pkl")

In [26]:
get_best_performance

Experiment,Id,Type,Status,Details Page,Docs Page
online_news_project,HD_74fb10e2-6617-418a-87ba-0fe657f41206_29,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Model Deployment

Create an inference config and deploy the model as a web service.

In [38]:
script_file_name = './score.py'
inference_config = InferenceConfig(entry_script=script_file_name)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 2, 
                                               memory_gb = 4, 
                                               tags = {'Company': "Mashable", 'type': "Hyperdrive"}, 
                                               description = 'sample service for Capstone Project Hyperdrive Classifier for Online News popularity')
aci_service_name = 'hyperdrive-deployment'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)
print(aci_service.scoring_uri)
print(aci_service.swagger_uri)

hyperdrive-deployment
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...........................
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 9f1344a1-c80d-4850-a562-532a2ac5a6b2
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, FileNotFoundError: [Errno 2] No such file or directory: 'azureml-models/best_model/1/model.pkl', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, FileNotFoundError: [Errno 2] No such file or directory: 'azureml-models/best_model/1/model.pkl', please run print(service.get_logs()) to get details."
    }
  ]
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 9f1344a1-c80d-4850-a562-532a2ac5a6b2
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, FileNotFoundError: [Errno 2] No such file or directory: 'azureml-models/best_model/1/model.pkl', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, FileNotFoundError: [Errno 2] No such file or directory: 'azureml-models/best_model/1/model.pkl', please run print(service.get_logs()) to get details."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Failed\nOperation ID: 9f1344a1-c80d-485

In [ ]:
aci_service.get_logs()

In [ ]:
test_data = pd.concat([X_test, y_test], axis=1)
test_data = test_data[10:15]
display(test_data)